Step 1: Importing the libraries

In [1]:
import numpy as np
import pandas as pd

Step 2: Importing dataset

In [ ]:
dataset = pd.read_csv('Data.csv')
X = dataset.iloc[ : , :-1].values
Y = dataset.iloc[ : , 3].values

#  -----------------------------------------------------------

xl = pd.ExcelFile("dognition_data_aggregated_by_dogid.xlsx")
# pd.read_excel()
# xlrd is used by pandas
xl.sheet_names
print 'Sheet name of the excel file:', xl.sheet_names[0]
df = xl.parse("dog_id_max_ranks")
# produce a dataframe
# df.head()


# For each column, show the number of nan values

# Search for the nan values in each column
print df.isnull().sum()
# df.isnull() is also a dataframe, and then sum() by columns


# Search for the nan values in each row

num = np.sum(df.isnull().values, axis = 1).tolist()
l1 = list(set(num))
l1.sort()
fre = {i:num.count(i) for i in l1}
dt = pd.DataFrame(fre.items(), columns=['Number of NaNs', 'Count'])
print dt


Step 3: Handling the missing data and outliers

In [ ]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values = "NaN", strategy = "mean", axis = 0)
imputer = imputer.fit(X[ : , 1:3])
X[ : , 1:3] = imputer.transform(X[ : , 1:3])

# ----------------------------------------------------------

db = df.drop(df.index[[num.index(22)]])
# Simply handling the missing data


Step 4: Encoding categorical data

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[ : , 0] = labelencoder_X.fit_transform(X[ : , 0])

# Creating a dummy variable
# a dummy variable is a categorical variable using 0 and 1

onehotencoder = OneHotEncoder(categorical_features = [0])
X = onehotencoder.fit_transform(X).toarray()
labelencoder_Y = LabelEncoder()
Y =  labelencoder_Y.fit_transform(Y)

# ----------------------------------------------------------------

db2 = db.copy()
for y in db2.columns:
    if db2[y].dtype == object:
        db2[y] = db2[y].astype('category')
db2['Dog_Fixed'] = db2['Dog_Fixed'].astype('category') 
db2['DNA_Tested'] = db2['DNA_Tested'].astype('category')
db2['Subscribed'] = db2['Subscribed'].astype('category')
# categorical data in pandas is like factor data in R

# change unreasonable values in the data

db2.loc[db2['Weight'] == 0.0,'Weight'] = 0.1
# first check the distribution of Weight, and other int and float columns

db2.loc[db2['Max_Dogs'] == 0.0, 'Max_Dogs'] = 1.0
# first check the distribution of Max_Dogs

# convert the column below to ordered category

db2['Last_Active_At'] = db2['Last_Active_At'].cat.as_ordered()
print db2.dtypes
print

print 'The columns of ordered category:'
for i in db2.columns:
    if db2[i].dtype.name == 'category' and db2[i].cat.ordered == True:
        print i


Step 5: Splitting the datasets into training sets and Test sets

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split( X , Y , test_size = 0.2, random_state = 0)

Step 6: Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.fit_transform(X_test)

#-------------------------------------------------------

from ggplot import *

p = ggplot(db2, aes(x='Total Tests Completed'))
# p + geom_histogram()
# p + geom_histogram(binwidth=1)
p + geom_histogram(bins=20)

p = ggplot(db2, aes(x='Gender',y='Total Tests Completed'))
# p + geom_violin(alpha = .75)
p + geom_violin()

import seaborn as sns
# seaborn.__version__
sns.violinplot(x=db2['Gender'], y=db2['Total Tests Completed'], inner=None, color="white", 
               cut=0)
sns.stripplot(x=db2['Gender'], y=db2['Total Tests Completed'], jitter=.3,  color="black", 
              alpha=.1, size=4)

sns.set_style("whitegrid")
g=sns.boxplot(x=db2['Total Tests Completed'])

temp = np.log10(db2['Total Tests Completed'])
sns.boxplot(x=temp)

sns.boxplot(x="Gender", y="Total Tests Completed", data=db2)
